# Section 2: IntegratedML

In this section, you will be running code blocks to train machine learning models. Each code block has some explanatory context above it, which you can refer back to if need be. 

Otherwise, we will explain each of the code blocks out loud, so you don't need to read all of the text.

## Part A: Getting ready for SQL Queries

You will be able to do this whole exercise in the management portal of your CloudSQL instance. 

But to take it easy on the cutting, pasting, and typing, we will run the SQL queries from this
jupyter notebook.  

For this section simply run the cells but don't worry too much about what they are doing. At the end
of this section, you will have a function called **query** that you can use to issue your SQL commands.

In [1]:
!pip install sqlparse

import pandas as pd
import iris
import warnings
import pdb
import sqlparse
from sklearn.metrics import roc_auc_score
import getpass

warnings.simplefilter("ignore")

In [2]:
# connect to iris 

hostname = "iris"
port = 1972
namespace = "USER"
username = "SuperUser"
password = "SYS"

# connect
connection = iris.connect("{:}:{:}/{:}".format(hostname, port, namespace), username, password)

In [3]:
# uncomment out this cell, if you are connecting directly to your cloud instance.

# on a mac, select the whole cell and hit command + /

# change your hostname to the deployment and when you run the cell, it'll ask for your password.

# # change these variables to reflect your connection

# hostname = "k8s-a3174c48-a54f71ab-1502f8dc12-258b58985f585303.elb.us-east-1.amazonaws.com"
# port = 1972

# namespace = "USER"
# username = "SQLAdmin"
# password = getpass.getpass("Password: ")

# # connect
# connection = iris.connect("{:}:{:}/{:}".format(hostname, port, namespace), username, password)

In [4]:
def query(sql):
    statements = sqlparse.parse(sql)
    assert len(statements) == 1
    if statements[0].get_type() == 'SELECT':
        return pd.read_sql_query(sql, connection, dtype = None)
    else:
        cursor = connection.cursor()
        result = cursor.execute(sql)

In [5]:
# do some clean up
query("DROP VIEW IF EXISTS SQLUser.stroke_train")
query("DROP VIEW IF EXISTS SQLUser.stroke_eval")
query("DROP MODEL IF EXISTS StrokeModel")

In [6]:
query("DROP VIEW IF EXISTS SQLUser.stroke_data_processed")
query('''

CREATE VIEW stroke_data_processed AS
SELECT
a.%ID as iris_id,
a.id, 
a.gender, 
a.date_birth, 
DATEDIFF('yyyy', a.date_birth,'12/31/2020') AS age, 
a.hypertension, 
a.heart_disease, 
a.ever_married, 
a.work_type, 
a.zip,
b.Residence_type, 
a.avg_glucose_level, 
a.height_in, 
a.weight_lb, 
CASE 
   WHEN a.weight_lb = 0
   THEN NULL
   ELSE ROUND(a.weight_lb/SQUARE(a.height_in)*703,1)
END AS bmi,
a.smoking_status, 
a.stroke
FROM stroke_data_raw AS a 
JOIN zip_codes AS b 
ON a.zip = b.zip

''')


## Part B: Splitting our data into training and validation rows

Let's start by taking a look at where you ended Marta's section.  You should have a single table now
with data from the stroke table and the zip codes table joined together.

And you should have some useful derived columns like patient age, and bmi that may be good predictors
of getting a stroke.


In [7]:
query('''

SELECT * FROM SQLUser.stroke_data_processed

''')

,iris_id,id,gender,date_birth,age,hypertension,heart_disease,ever_married,work_type,zip,Residence_type,avg_glucose_level,height_in,weight_lb,bmi,smoking_status,stroke
0,1,77,Female,4/5/2007,13,0,0,No,children,98671,Rural,85.81,60.2,95.88,18.6,Unknown,0
1,2,84,Male,5/17/1965,55,0,0,Yes,Private,99126,Urban,89.17,72.8,237.48,31.5,never smoked,0
2,3,91,Female,3/29/1978,42,0,0,No,Private,98382,Urban,98.53,66.9,117.78,18.5,never smoked,0
3,4,99,Female,12/24/1989,31,0,0,No,Private,98323,Urban,108.89,55.5,229.16,52.3,Unknown,0
4,5,129,Female,8/23/1996,24,0,0,No,Private,98465,Urban,97.55,61.8,142.34,26.2,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,4905,72911,Female,8/18/1963,57,1,0,Yes,Private,98662,Rural,129.54,75.6,495.11,60.9,smokes,0
4905,4906,72914,Female,5/10/2001,19,0,0,No,Private,98390,Urban,90.57,64.6,143.66,24.2,Unknown,0
4906,4907,72915,Female,11/18/1975,45,0,0,Yes,Private,98320,Urban,172.33,77.2,384.04,45.3,formerly smoked,0
4907,4908,72918,Female,4/18/1967,53,1,0,Yes,Private,98822,Urban,62.55,61.0,160.38,30.3,Unknown,1


Now let us use the iris_id column to split the data.  

We'd like 80% of the data for training and 20% of the data for validation.  

So assuming the last digit of the iris_id is uncorrelated with any information in it's row, we can use
that.  If it ends in 0 or 1, we'll call it validation data (the 20%) and if it's greater than 1, we'll
use it for training (the 80%).  You can get the last digit of something by MOD(iris_id, 10) as the MOD operator is the remainder operator - it will take the remainder after dividing by 10, which is a way
to get the last digit.  

In [8]:
query('''

CREATE VIEW stroke_eval AS
SELECT * FROM stroke_data_processed
WHERE MOD(iris_id, 10) in (0, 1)

''')


Let's check if it's the right number of rows 

In [9]:
query("SELECT COUNT(*) FROM stroke_eval")

,Aggregate_1
0,981


In [10]:
query('''

CREATE VIEW stroke_train AS
SELECT * FROM stroke_data_processed
WHERE MOD(iris_id, 10) > 1

''')
query("SELECT COUNT(*) FROM stroke_train")

,Aggregate_1
0,3928


In [11]:
query("SELECT COUNT(*) FROM stroke_eval")

,Aggregate_1
0,981


## Part C: Using IntegratedML to create, train, validate, and use a model

### CREATE MODEL

The CREATE MODEL statement will create the model.  

It needs to know the name of the model, the data that you will use to train it, and the column that 
represents the column that you will train on.  All the other columns are assumed to be the inputs
(features) and the prediction column is often called the label.  


In [12]:
query('''

CREATE MODEL StrokeModel predicting(Stroke) FROM stroke_train

''')

### TRAIN MODEL

Now that we have a model, we need to train it using the TRAIN MODEL statement.  

This statement needs to know the name of the model and a new name for the trained model.

We are also passing in a seed argument so that everyone in the class should get the same predictable
results.  This is an optional argument.  

The training itself will take about a minute. Please patient!

In [13]:
query('''

TRAIN MODEL StrokeModel AS StrokeModelTrained USING {"seed": 42}

''')

### VALIDATE MODEL

Now that we have a trained model, we need to figure out if the model is any good at predicting 
whether a patient will get a stroke.  

To do this, we will validate the model using the evaluation data. Remember this is data that the training process has not seen, so it will be an accurate view as to have accurate our model us.  

The VALIDATE MODEL statement takes a model, a name for the validation run, a trained model, and a source of data that it should use for the validation itself.  

In [14]:
query('''

VALIDATE MODEL StrokeModel AS StrokeValidation USE StrokeModelTrained FROM stroke_eval

''')

The VALIDATE MODEL statement will run the validation, but now we have to find the metrics and take a look at them.  You can do this by looking at the table INFORMATION_SCHEMA.ML_VALIDATION_METRICS.  Here you will see all the validation runs that you have run. Since it's only one, we can do a SELECT *.

In [15]:
query('''

SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS

''')

,MODEL_NAME,TRAINED_MODEL_NAME,VALIDATION_RUN_NAME,METRIC_NAME,METRIC_VALUE,TARGET_VALUE
0,StrokeModel,StrokeModelTrained,StrokeValidation,Precision,0.16,1.0
1,StrokeModel,StrokeModelTrained,StrokeValidation,Recall,0.57,1.0
2,StrokeModel,StrokeModelTrained,StrokeValidation,F-Measure,0.25,1.0
3,StrokeModel,StrokeModelTrained,StrokeValidation,Accuracy,0.82,NaN


You can see a number of metrics in that table.  This is a 'needle in a haystack' problem, meaning the number of positive instances (patients with stroke) is very small. This makes the accuracy metric almost useless.  

The most widely used metrics for a problem like this are **Recall**, **Precision**, and **AUC** short for Area Under Curve, in particular the Receiver Operator Characteristic curve.  

**Recall** is the # of true stroke patients that you correctly predicted divided by the total number of true stroke patients. You can think of it as the number of stroke patients that you **recalled** from the group.

**Precision** is closely related. This is the # of true stroke patients that you correctly predicted divided by the number of stroke predictions that you made. You can think of as how **precise** your stroke guess were - what % of the time were you right?

**AUC** is the most complicated but is a very popular metric because it does not depend on where you set your threshold. If you imagine your predictions ordered by highest probability to lowest probability, you would want your stroke patients at the top of that list. AUC is a measure of how well your model puts the stroke patients near the top of the list.  

### SELECT PREDICT, PROBABILITY

Now let make some predictions to make sure everything is working well.

In [16]:
query('''

SELECT 
    TOP(100)
    PREDICT(StrokeModel use StrokeModelTrained) as prediction,
    stroke,
    PROBABILITY(StrokeModel use StrokeModelTrained) as probability_stroke, 
    *
FROM 
    SQLUser.stroke_eval

''')


,prediction,stroke,probability_stroke,iris_id,id,gender,date_birth,age,hypertension,heart_disease,ever_married,work_type,zip,Residence_type,avg_glucose_level,height_in,weight_lb,bmi,smoking_status,stroke
0,0,0,0.009934,81,1329,Female,10/27/1977,43,0,0,No,Govt_job,98001,Rural,101.35,70.5,227.66,32.2,never smoked,0
1,0,0,0.008044,2700,41191,Male,12/30/1980,40,0,0,Yes,Private,98001,Rural,64.84,72.0,196.15,26.6,never smoked,0
2,0,0,0.016738,3211,48759,Female,6/28/1975,45,0,0,Yes,Private,98001,Rural,176.48,69.7,165.85,24.0,formerly smoked,0
3,0,0,0.001358,3631,54344,Female,3/1/2008,12,0,0,No,children,98001,Rural,80.89,59.1,99.87,20.1,Unknown,0
4,0,0,0.004222,3851,57347,Female,7/16/1991,29,0,0,No,Govt_job,98001,Rural,57.02,60.2,221.67,43.0,formerly smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0.271303,2150,32884,Female,12/1/1940,80,1,0,Yes,Private,98059,Urban,210.96,74.4,250.39,31.8,never smoked,0
96,0,0,0.002607,1610,24727,Male,8/3/2000,20,0,0,No,Private,98070,Rural,117.98,74.4,243.30,30.9,smokes,0
97,0,0,0.004378,3740,56089,Female,5/14/1995,25,0,0,Yes,Private,98070,Rural,63.64,65.0,188.11,31.3,formerly smoked,0
98,0,0,0.005896,4011,59872,Female,4/30/1982,38,0,0,Yes,Private,98070,Rural,80.82,63.0,278.34,49.3,never smoked,0


### Area Under the Curve

Great.  We have a model that can make some predictions. It has this performance:

Recall = 0.35 (it found 35% of the stroke patients)

Precision = 0.19 (When it predicted stroke, it was right 19% of the time)

Unfortunately at the time of this writing, our validation metrics do not calculate **AUC**.  This is an enhancement request that the team is working on.  In the meantime, if we have all of our predictions, we can use a python library to easily calculate it.  Let's do that.


In [17]:
df = query('''

SELECT 
    stroke,
    PROBABILITY(StrokeModel use StrokeModelTrained) as probability_stroke
FROM 
    SQLUser.stroke_eval

''')
roc_auc_score(df['stroke'], df['probability_stroke'])

0.8132616487455196

We can see that using IntegratedML gives us an AUC = 0.81.  You can think of AUC as a substitute for 'accuracy'. So at 81% accurate, this model is pretty good.  But it could probably be better. For the next exercise, we will be using Jupyter Notebook and Python libraries to hand build some ML models to see if we can beat that 0.81.  Good luck!


### Let's Build a Better Model